## Use mobienet for aesthetic feature extraction 

In [1]:
from keras.models import Model
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
import numpy as np

mobilenet = MobileNet(input_shape=(224, 224, 3), weights=None, include_top=False, pooling='avg')
mobilenet.load_weights("./model/aesthetic_mobilenet_without_top.h5")
mobilenet.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [2]:
import lmdb
import os
import cPickle as pickle
from tqdm import tqdm

batch_size = 128
img_path = "../dataset/AVA/imgs/"

env = lmdb.open("./data/aesthetic_features.lmdb", map_size=1024*8*300000)
ALL_IMGIDS = set()

def img_preprocess(imgfile):
    try:
        filename = img_path+imgfile
        img = image.load_img(filename, target_size=(224, 224))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        return x
    except Exception, e:
        return None
    
def save_to_lmdb(imgids, features):
    txn = env.begin(write=True)
    for _imgid, _feature in zip(imgids, features):
        str_feature = _feature.tostring()
        txn.put(_imgid, str_feature)
    txn.commit()
    

batch_imgids = []
batch_X = []
for imgfile in tqdm(os.listdir(img_path)):
    x = img_preprocess(imgfile)
    imgid = os.path.splitext(imgfile)[0]
    if x is None:
        continue
    batch_X.append(x)
    batch_imgids.append(imgid)
    ALL_IMGIDS.add(imgid)
    if len(batch_X) == batch_size:
        batch_features = mobilenet.predict_on_batch(np.array(batch_X))
        save_to_lmdb(batch_imgids, batch_features)
        batch_imgids = []
        batch_X = []
        
if len(batch_X) > 0:
    batch_features = mobilenet.predict_on_batch(np.array(batch_X))
    save_to_lmdb(batch_imgids, batch_features)

env.close()

100%|██████████| 254782/254782 [38:36<00:00, 109.99it/s]


In [3]:
len(ALL_IMGIDS)

254778